In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import cv2
import os
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Model,Sequential, Input, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, AveragePooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.applications import DenseNet201

In [ ]:
from google.colab import drive
drive.mount('/content/drive')   

Mounted at /content/drive


In [ ]:
!unzip -qq /content/drive/'My Drive'/Code/tomatoes.zip

In [ ]:
dataset_path = '/content/PlantVillage'
 
class_name = []
images = []
labels = []
for index, folder in enumerate(os.listdir(dataset_path)):
  class_name.append(folder)
  image_path = os.path.join(dataset_path, folder)
  print(image_path, len(os.listdir(image_path)))
  for image in os.listdir(image_path):
    labels.append(index)
    images.append(os.path.join(dataset_path, folder, image))
 
X = []
for index, path in enumerate(tqdm(images)):
  try:
    X.append(cv2.resize(cv2.imread(path),dsize=(256,256)))
  except:
    labels.pop(index)
X = np.array(X)
y = np.array(labels)
Y = to_categorical(y, num_classes=len(class_name))

  0%|          | 0/12380 [00:00<?, ?it/s]

/content/PlantVillage/Tomato_Bacterial_spot 2127
/content/PlantVillage/Tomato_Septoria_leaf_spot 2362
/content/PlantVillage/Tomato__Tomato_YellowLeaf__Curl_Virus 3209
/content/PlantVillage/Tomato_Late_blight 2499
/content/PlantVillage/Tomato_healthy 2183


100%|██████████| 12380/12380 [00:41<00:00, 297.81it/s]


In [ ]:
print("[INFO] Spliting data to train, val, test")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4,stratify = Y, shuffle=True, random_state=100)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify = y_test, shuffle=True, random_state=100)
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")
aug_val = ImageDataGenerator(rescale=1./255)


[INFO] Spliting data to train, val, test


In [ ]:
print(X_train.shape)
print(X_val.shape)

(8663, 256, 256, 3)
(1856, 256, 256, 3)


In [ ]:
def build_densenet():
    densenet = DenseNet201(weights='imagenet', include_top=False)
    
    input = Input(shape=(256, 256, 3))
    x = Conv2D(3, (3, 3), padding='same')(input)
    x = densenet(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(5,activation = 'softmax', name='root')(x)

    # model
    model = Model(input,output)
    
    optimizer = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=0.1, decay=0.0)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
filepath="/content/weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]
model = build_densenet()

history=model.fit(aug.flow(X_train, y_train, batch_size=32),
                               epochs=25, steps_per_epoch=len(X_train)//32,
                               validation_data=aug.flow(X_val,y_val,
                               batch_size=64), callbacks=callbacks_list)

74842112/74836368 [==============================] - 0s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 3)       84        
_________________________________________________________________
densenet201 (Functional)     (None, None, None, 1920)  18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1920)              7680      
_________________________________________________________________
dropout (Dropout)            (None, 1920)              0         
_______________________________________________________

KeyboardInterrupt: ignored

In [ ]:
from keras.models import load_model
from sklearn.metrics import classification_report

for i in range(1,10):
  modellink = '/content/weights-' + '0' + str(i) + '.hdf5'
  model = load_model(modellink)
  #y_pred = model.predict(X_test, batch_size=64, verbose=1)
  #y_pred_bool = np.argmax(y_pred, axis=1)
  #print(classification_report(y_test, y_pred_bool))
  scores = model.evaluate(X_test, y_test)
  print(f"Test Accuracy: {scores[1]*100}")

for i in range(10,26):
  modellink = '/content/weights-' + str(i) + '.hdf5' 
  model = load_model(modellink)
  scores = model.evaluate(X_test, y_test)
  print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accuracy) + 1)
#Train and validation accuracy
plt.plot(epochs, accuracy, 'b', label='Training accurarcy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
pic = '/content/drive/MyDrive/Code/Thư mục không có tiêu đề/New folder/benh/Late.jpg'
#model = load_model('/content/weights-24.hdf5')
X = []
X.append(cv2.resize(cv2.imread(pic),dsize=(256,256)))
X = np.array(X)
predictions = model.predict(X)
print(predictions)

In [ ]:
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
img_path = r'/content/PlantVillage/Tomato_Septoria_leaf_spot/962119e0-4165-4c4e-bf7e-fd68f89e4e6b___Keller.St_CG 2016.JPG'
img = load_img(img_path)
data = img_to_array(img)
samples = expand_dims(data, 0)
aug = ImageDataGenerator(
    zoom_range=[0.5,1.0])
it = aug.flow(samples, batch_size=1)
#pyplot.imshow(img)
for i in range(3,6):
	pyplot.subplot(330 + 1 + i)
	batch = it.next()
	image = batch[0].astype('uint8')
	pyplot.imshow(image)
pyplot.show()

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import load_model
#weights_path = "/content/drive/MyDrive/Code/Thư mục không có tiêu đề/weights-24.hdf5"
#model = load_model(weights_path)

y_hat = model.predict(X_test)
y_pred = np.argmax(y_hat, axis=1)
y_test_label =  np.argmax(y_test, axis=1)
 
# Tính accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test_label, y_pred)
print('Accuracy: %f' % accuracy)
# Tính precision tp / (tp + fp)
precision = precision_score(y_test_label, y_pred, average='macro')
print('Precision: %f' % precision)
# Tính recall: tp / (tp + fn)
recall = recall_score(y_test_label, y_pred, average='macro')
print('Recall: %f' % recall)
# Tính f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test_label, y_pred, average='macro')
print('F1 score: %f' % f1)
# Tính Area under ROC
auc = roc_auc_score(y_test, y_hat, multi_class='ovr')
print('ROC AUC: %f' % auc)
# Tính confusion matrix
matrix = confusion_matrix(y_test_label, y_pred)
print(matrix)

In [ ]:
print(y_test.shape)

(1857, 5)
